In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/cameronpaschall/Desktop/Instacart Basket Analysis'

In [3]:
# Import data set
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_new_columns.pkl'))

In [4]:
df = ords_prods_merge

## Step 2: Aggregated Mean

In [6]:
df.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
1.0              15.457838
2.0              17.277920
3.0              17.170395
4.0              17.811403
5.0              15.215751
6.0              16.439806
7.0              17.225802
8.0              15.340650
9.0              15.895474
10.0             20.197148
11.0             16.170638
12.0             15.887671
13.0             16.583536
14.0             16.773669
15.0             16.165037
16.0             17.665606
17.0             15.694469
18.0             19.310397
19.0             17.177343
20.0             16.473447
21.0             22.902379

## Step 3: Analyze the result

The means for the entire df are slightly different than those of the subset. There are a lot more entries in the entire df so this was to be expected.

## Step 4: Creating Flag

In [7]:
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [9]:
pd.options.display.max_rows = None

In [11]:
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
df['loyalty_flag'].value_counts(dropna = False)

Regular customer    15891507
Loyal customer      10294027
New customer         6249525
Name: loyalty_flag, dtype: int64

## Step 5: Spending Habits

In [13]:
## groupby function to see stats of customer spending 
df.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [14]:
## New customers spend the most on average while loyal customers seem to have refined their shopping habits and spend the lowest on average

## Step 6: Create spending flag

In [15]:
## Create new column
df['spending'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
df.head(5)

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   
3                    1.11       26088                  4          0   
4                    1.11       26405                  5          0   

                              product_name  ...  department_id  prices  \
0                                     Soda  ...            7.0     9.0   
1  Organic Unsweetened Vanilla Almond Milk  ...           16.0    12.5   
2                      Original Beef Jerky  ...           19.0     4.4   
3               Aged White Cheddar Popcorn  ...           19.0     4.7   
4         XL Pick-A-Size Paper Towel Rolls  ...           17.0     1.0   

   _merge    price_range_loc    Busiest days    Slowest days  \
0    both  Mid-range product  Regularly busy  Regularly busy   
1    both  Mid-range product  Regularly busy  Regularly busy   
2    both  Low-range product  Regularly busy  Regularly busy   
3    both  Low-range product  Regularly busy  Regularly busy   
4    both  Low-range product  Regularly busy  Regularly busy   

  busiest_period_of_day max_order  loyalty_flag  spending  
0        Average orders        10  New customer  6.367797  
1        Average orders        10  New customer  6.367797  
2        Average orders        10  New customer  6.367797  
3        Average orders        10  New customer  6.367797  
4        Average orders        10  New customer  6.367797  

[5 rows x 21 columns]

In [18]:
## Create flag for over or under $10
df.loc[df['spending'] < 10, 'type_of_spender'] = 'Low Spender'
df.loc[df['spending'] >= 10, 'type_of_spender'] = 'High Spender'

In [20]:
df['type_of_spender'].value_counts(dropna = False)

Low Spender     31800414
High Spender      634645
Name: type_of_spender, dtype: int64

In [19]:
df.head(5)

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   
3                    1.11       26088                  4          0   
4                    1.11       26405                  5          0   

                              product_name  ...  prices  _merge  \
0                                     Soda  ...     9.0    both   
1  Organic Unsweetened Vanilla Almond Milk  ...    12.5    both   
2                      Original Beef Jerky  ...     4.4    both   
3               Aged White Cheddar Popcorn  ...     4.7    both   
4         XL Pick-A-Size Paper Towel Rolls  ...     1.0    both   

     price_range_loc    Busiest days    Slowest days busiest_period_of_day  \
0  Mid-range product  Regularly busy  Regularly busy        Average orders   
1  Mid-range product  Regularly busy  Regularly busy        Average orders   
2  Low-range product  Regularly busy  Regularly busy        Average orders   
3  Low-range product  Regularly busy  Regularly busy        Average orders   
4  Low-range product  Regularly busy  Regularly busy        Average orders   

  max_order  loyalty_flag  spending type_of_spender  
0        10  New customer  6.367797     Low Spender  
1        10  New customer  6.367797     Low Spender  
2        10  New customer  6.367797     Low Spender  
3        10  New customer  6.367797     Low Spender  
4        10  New customer  6.367797     Low Spender  

[5 rows x 22 columns]

## Step 7: Frequency Flag

In [21]:
df.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_time_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'Busiest days',
       'Slowest days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'spending', 'type_of_spender'],
      dtype='object')

In [22]:
## Create new column
df['order_freq'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [24]:
df.head(3)

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   

                              product_name  ...  _merge    price_range_loc  \
0                                     Soda  ...    both  Mid-range product   
1  Organic Unsweetened Vanilla Almond Milk  ...    both  Mid-range product   
2                      Original Beef Jerky  ...    both  Low-range product   

     Busiest days    Slowest days busiest_period_of_day max_order  \
0  Regularly busy  Regularly busy        Average orders        10   
1  Regularly busy  Regularly busy        Average orders        10   
2  Regularly busy  Regularly busy        Average orders        10   

   loyalty_flag  spending  type_of_spender order_freq  
0  New customer  6.367797      Low Spender       20.0  
1  New customer  6.367797      Low Spender       20.0  
2  New customer  6.367797      Low Spender       20.0  

[3 rows x 23 columns]

In [25]:
df.loc[df['order_freq'] > 20, 'frequency_flag'] = 'Non-frequent Customer'
df.loc[(df['order_freq'] > 10) & (df['order_freq'] <= 20), 'frequency_flag'] = 'Regular Customer'
df.loc[df['order_freq'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [26]:
df.head(3)

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   

                              product_name  ...    price_range_loc  \
0                                     Soda  ...  Mid-range product   
1  Organic Unsweetened Vanilla Almond Milk  ...  Mid-range product   
2                      Original Beef Jerky  ...  Low-range product   

     Busiest days    Slowest days busiest_period_of_day max_order  \
0  Regularly busy  Regularly busy        Average orders        10   
1  Regularly busy  Regularly busy        Average orders        10   
2  Regularly busy  Regularly busy        Average orders        10   

   loyalty_flag  spending type_of_spender  order_freq    frequency_flag  
0  New customer  6.367797     Low Spender        20.0  Regular Customer  
1  New customer  6.367797     Low Spender        20.0  Regular Customer  
2  New customer  6.367797     Low Spender        20.0  Regular Customer  

[3 rows x 24 columns]

In [27]:
df['frequency_flag'].value_counts(dropna = False)

Frequent Customer        22815145
Regular Customer          6929908
Non-frequent Customer     2690006
Name: frequency_flag, dtype: int64

## Step 9: Export as pickle

In [28]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_flags.pkl'))